In [ ]:
%pip install langchain
%pip install langchain-deepseek
%pip install langchain-ollama
%pip install python-dotenv

In [ ]:

from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env",override=True)
from os import getenv

key = getenv("DEEPSEEK_API_KEY") 
url = getenv("DEEPSEEK_API_BASE")
model = getenv("DEEPSEEK_MODEL")
rmodel = getenv("DEEPSEEK_RMODEL")

print(key, url, model)

In [ ]:
from langchain_deepseek import ChatDeepSeek


llm = ChatDeepSeek(
    model=model,
    api_base=url,
    api_key=key
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="deepseek-r1:latest"
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg)

In [ ]:
import os

key = getenv("SF_API_KEY") 
url = getenv("SF_API_BASE")
model = getenv("SF_MODEL")
rmodel = getenv("SF_RMODEL")

os.environ["DEEPSEEK_API_KEY"] = key
os.environ["DEEPSEEK_API_BASE"] = url
print(key, url, model)

In [ ]:
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model=model,
    api_base=url,
    api_key=key
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to chinese. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_deepseek import ChatDeepSeek


llm = ChatDeepSeek(
    model=model,
    api_base=url,
    api_key=key
)
messages = [
    SystemMessage("Translate the following from English into Chinese."),
    HumanMessage("hi!"),
]

print(llm.invoke(messages).content)

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


llm = ChatDeepSeek(
    model=model,
    api_base=url,
    api_key=key
)

system_template = "Translate the following from English into {language}"
human_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), 
     ("user", human_template)]
)

chain = prompt_template | llm | StrOutputParser()
# print(chain)
print(chain.invoke({"language": "Chinese", "text": "hi!"}))


In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser


llm = ChatDeepSeek(
    model=model,
    api_base=url,
    api_key=key
)

system_template = SystemMessagePromptTemplate.from_template( "Translate the following from English into {language}")
human_template = HumanMessagePromptTemplate.from_template("{text}")

prompt_template = ChatPromptTemplate.from_messages(
    [system_template, 
     human_template]
)

chain = prompt_template | llm | StrOutputParser()
# print(chain)
print(chain.invoke({"language": "Chinese", "text": "hi!"}))

In [ ]:
# 定义我们想要接收的数据格式
from pydantic import BaseModel, Field

from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

class FlowerDescription(BaseModel):
    flower_type: str = Field(description="鲜花的种类")
    description: str = Field(description="鲜花的描述文案")

# 创建输出解析器
output_parser = PydanticOutputParser(pydantic_object=FlowerDescription)

# 获取输出格式指示
format_instructions = output_parser.get_format_instructions()
# 打印提示
# print("输出格式：",format_instructions)

# 创建提示模板
prompt_template = ChatPromptTemplate.from_template("""您是一位专业的鲜花店文案撰写员。
对于{flower} ，您能提供一个吸引人的简短中文描述吗？
{format_instructions}""")

llm = ChatDeepSeek(
    model=model,
    api_key=key,
    api_base=url,
)

chain = prompt_template | llm | output_parser
out = chain.invoke({"flower":"玫瑰", "format_instructions":format_instructions})
print(out.model_dump_json())

In [ ]:
# 导入所需要的库和模块
from langchain_deepseek import ChatDeepSeek
from langchain.output_parsers import PydanticOutputParser,OutputFixingParser
from pydantic import BaseModel, Field
from typing import List

# 使用Pydantic创建一个数据格式，表示花
class Flower(BaseModel):
    name: str = Field(description="name of a flower")
    colors: List[str] = Field(description="the colors of this flower")
# 定义一个用于获取某种花的颜色列表的查询
flower_query = "Generate the charaters for a random flower."

# 定义一个格式不正确的输出
misformatted = "{'name': '康乃馨', 'colors': ['粉红色','白色','红色','紫色','黄色']}"

# 创建一个用于解析输出的Pydantic解析器，此处希望解析为Flower格式
parser = PydanticOutputParser(pydantic_object=Flower)
# 使用Pydantic解析器解析不正确的输出

llm = ChatDeepSeek(
    model=model,
    api_key=key,
    api_base=url,
)
# 使用OutputFixingParser创建一个新的解析器，该解析器能够纠正格式不正确的输出
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# 使用新的解析器解析不正确的输出
result = fix_parser.parse(misformatted) # 错误被自动修正
print(result) # 打印解析后的输出结果

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)
input_data = "What is the capital of France?"
summary_chain = (
    ChatPromptTemplate.from_template("Summarize this: {input}")
    | llm
    | StrOutputParser()
)
sentiment_chain = (
    ChatPromptTemplate.from_template("What is the sentiment of this: {input}")
    | llm
    | StrOutputParser()
)

parallel_chain = RunnableParallel({
    "summary": summary_chain,
    "sentiment": sentiment_chain
})
parallel_result = parallel_chain.invoke({"input": input_data})
print(parallel_result)

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch


llm = ChatDeepSeek(
    model=model,
    api_key=key,
    api_base=url
)

# 定义子链
tech_prompt = ChatPromptTemplate.from_template(
    "作为科技作者，用专业术语解释：{concept}"
)
sports_prompt = ChatPromptTemplate.from_template(
    "作为体育解说员，用生动语言描述：{concept}"
)
general_prompt = ChatPromptTemplate.from_template(
    "用通俗语言解释：{concept}"
)

tech_chain = tech_prompt | llm | StrOutputParser()
sports_chain = sports_prompt | llm | StrOutputParser()
general_chain = general_prompt | llm | StrOutputParser()

# 构建分支逻辑
branch_chain = RunnableBranch(
    (lambda x: x["topic"] == "科技", tech_chain),
    (lambda x: x["topic"] == "体育", sports_chain),
    general_chain
)

# 完整执行链
# full_chain = RunnableParallel({  # 并行处理输入
#     "concept": lambda x: x["concept"],
#     "topic": lambda x: x["topic"]
# }) | branch_chain
full_chain = branch_chain

print("***********************")
print(full_chain.invoke({
    "concept": "神经网络", 
    "topic": "科技"
}))
print("***********************")

print(full_chain.invoke({
    "concept": "越位", 
    "topic": "体育"
}))

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains  import LLMChain, SequentialChain


# 第一个LLMChain：生成鲜花的介绍
llm = ChatDeepSeek(
    model=model,
    api_key=key,
    api_base=url,
)
template = """
你是一个植物学家。给定花的名称和类型，你需要为这种花写一个200字左右的介绍。
花名: {name}
颜色: {color}
植物学家: 这是关于上述花的介绍:"""
prompt_template = ChatPromptTemplate.from_template(template)
introduction_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_key="introduction"
)

# 第二个LLMChain：根据鲜花的介绍写出鲜花的评论
template = """
你是一位鲜花评论家。给定一种花的介绍，你需要为这种花写一篇200字左右的评论。
鲜花介绍:
{introduction}
花评人对上述花的评论:"""
prompt_template = ChatPromptTemplate.from_template(template)
review_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_key="review"
)

# 第三个LLMChain：根据鲜花的介绍和评论写出一篇自媒体的文案
template = """
你是一家花店的社交媒体经理。给定一种花的介绍和评论，你需要为这种花写一篇社交媒体的帖子，300字左右。
鲜花介绍:
{introduction}
花评人对上述花的评论:
{review}
社交媒体帖子:
"""
prompt_template = ChatPromptTemplate.from_template(template)
social_post_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_key="social_post_text"
)

# 总的链：按顺序运行三个链
overall_chain = SequentialChain(
    chains=[introduction_chain, review_chain, social_post_chain],
    input_variables=["name", "color"],
    output_variables=["introduction", "review", "social_post_text"],
    verbose=True
)

# 运行链并打印结果
result = overall_chain.invoke({
    "name": "玫瑰",
    "color": "黑色"
})
print(result)

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import (
    ConversationSummaryBufferMemory,
    ConversationBufferMemory
)

print(key,url)
# 初始化大语言模型
llm = ChatDeepSeek(
    model=model,
    api_key=key,
    api_base=url,
)

# 初始化对话链
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

# 第一天的对话
# 回合1
result = conversation("我姐姐明天要过生日，我需要一束生日花束。")
print(result)
# 回合2
# result = conversation("她喜欢粉色玫瑰，颜色是粉色的。")
# print("\n第二次对话后的记忆:\n", conversation.memory.buffer)
# print(result)

# 第二天的对话
# 回合3
# result = conversation("我又来了，还记得我昨天为什么要来买花吗？")
# print(result)

In [ ]:
%pip install langchain_experimental

In [ ]:
# 导入langchain的实用工具和相关的模块
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model=rmodel,
    api_key=key,
    api_base=url,
)

db = SQLDatabase.from_uri("sqlite:///test.db")

# 创建SQL数据库链实例，它允许我们使用LLM来查询SQL数据库
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

response = db_chain.run("数据库中有几张表")

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage

llm = ChatDeepSeek(
    model=rmodel,
    api_key=key,
    api_base=url,
)

llm.invoke([
    SystemMessage("Translate the following from English into Chinese."),
    HumanMessage("hi!"),
])